# Coordinate Alignment in linopy

linopy enforces strict defaults for coordinate alignment so that mismatches never silently produce wrong results.

| Operation | Shared-dim alignment | Extra dims on constant/RHS |
|-----------|---------------------|---------------------------|
| `+`, `-` | `"exact"` — must match | **Forbidden** |
| `*`, `/` | `"inner"` — intersection | Expands the expression |
| `<=`, `>=`, `==` | `"exact"` — must match | **Forbidden** |

**Why?** Addition and constraint RHS only change constant terms — expanding into new dimensions would duplicate the same variable. Multiplication changes coefficients, so expanding is meaningful. The rules are consistent: `a*x + b <= 0` and `a*x <= -b` always behave identically.

When coordinates don't match, use the named methods (`.add()`, `.sub()`, `.mul()`, `.div()`, `.le()`, `.ge()`, `.eq()`) with an explicit `join=` parameter.

Inspired by [pyoframe](https://github.com/Bravos-Power/pyoframe).

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import linopy

## What works by default

In [ ]:
m = linopy.Model()

time = pd.RangeIndex(5, name="time")
techs = pd.Index(["solar", "wind", "gas"], name="tech")

x = m.add_variables(lower=0, coords=[time], name="x")
y = m.add_variables(lower=0, coords=[time], name="y")
gen = m.add_variables(lower=0, coords=[time, techs], name="gen")

In [ ]:
# Addition/subtraction — matching coordinates
x + y

In [ ]:
# Multiplication — matching coordinates
factor = xr.DataArray([2, 3, 4, 5, 6], dims=["time"], coords={"time": time})
x * factor

In [ ]:
# Multiplication — partial overlap gives intersection
partial = xr.DataArray([10, 20, 30], dims=["time"], coords={"time": [0, 1, 2]})
x * partial  # result: time 0, 1, 2 only

In [ ]:
# Multiplication — different dims broadcast (expands the expression)
cost = xr.DataArray([1.0, 0.5, 3.0], dims=["tech"], coords={"tech": techs})
x * cost  # result: (time, tech)

In [ ]:
# Constraints — RHS with fewer dims broadcasts naturally
capacity = xr.DataArray([100, 80, 50], dims=["tech"], coords={"tech": techs})
m.add_constraints(gen <= capacity, name="cap")  # capacity broadcasts over time

## What raises an error

In [ ]:
# Addition with mismatched coordinates
y_short = m.add_variables(
    lower=0, coords=[pd.RangeIndex(3, name="time")], name="y_short"
)

try:
    x + y_short  # time coords don't match
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# Addition with extra dimensions on the constant
profile = xr.DataArray(
    np.ones((3, 5)), dims=["tech", "time"], coords={"tech": techs, "time": time}
)
try:
    x + profile  # would duplicate x[t] across techs
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# Multiplication with zero overlap
z = m.add_variables(lower=0, coords=[pd.RangeIndex(5, 10, name="time")], name="z")

try:
    z * factor  # z has time 5-9, factor has time 0-4 — no intersection
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# Constraint RHS with mismatched coordinates
partial_rhs = xr.DataArray([10, 20, 30], dims=["time"], coords={"time": [0, 1, 2]})

try:
    x <= partial_rhs
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# Constraint RHS with extra dimensions
w = m.add_variables(lower=0, coords=[techs], name="w")  # dims: (tech,)
rhs_2d = xr.DataArray(
    np.ones((5, 3)), dims=["time", "tech"], coords={"time": time, "tech": techs}
)
try:
    w <= rhs_2d  # would create redundant constraints on w[tech]
except ValueError as e:
    print("ValueError:", e)

## Positional alignment

A common pattern: two arrays with the same shape but different (or no) coordinate labels. The cleanest fix is to relabel one operand with `.assign_coords()` so that coordinates match explicitly:

In [ ]:
m2 = linopy.Model()

a = m2.add_variables(coords=[["x", "y", "z"]], name="a")
b = m2.add_variables(coords=[["p", "q", "r"]], name="b")

# Relabel b's coordinates to match a, then add normally
a + b.assign_coords(dim_0=a.coords["dim_0"])

In [ ]:
# Same for constraints
rhs = xr.DataArray([1.0, 2.0, 3.0], dims=["dim_0"], coords={"dim_0": ["p", "q", "r"]})
a <= rhs.assign_coords(dim_0=a.coords["dim_0"])

In [ ]:
# Shorthand: join="override" does the same (positional match, keeps left labels)
a.add(b, join="override")

## Other join modes

All named methods (`.add()`, `.sub()`, `.mul()`, `.div()`, `.le()`, `.ge()`, `.eq()`) accept a `join=` parameter:

| `join` | Coordinates kept | Fill |
|--------|-----------------|------|
| `"exact"` | Must match | `ValueError` if different |
| `"inner"` | Intersection | — |
| `"outer"` | Union | Zero (arithmetic) / NaN (constraints) |
| `"left"` | Left operand's | Zero / NaN for missing right |
| `"right"` | Right operand's | Zero for missing left |
| `"override"` | Left operand's | Positional alignment |

In [ ]:
i_a = pd.Index([0, 1, 2], name="i")
i_b = pd.Index([1, 2, 3], name="i")

a = m2.add_variables(coords=[i_a], name="a2")
b = m2.add_variables(coords=[i_b], name="b2")

print("inner:", list(a.add(b, join="inner").coords["i"].values))  # [1, 2]
print("outer:", list(a.add(b, join="outer").coords["i"].values))  # [0, 1, 2, 3]
print("left: ", list(a.add(b, join="left").coords["i"].values))  # [0, 1, 2]
print("right:", list(a.add(b, join="right").coords["i"].values))  # [1, 2, 3]

## Migrating from previous versions

Previous versions used a shape-dependent heuristic that caused silent bugs (positional alignment on same-shape operands, non-associative addition, broken multiplication). The new behavior:

| Condition | Old | New |
|-----------|-----|-----|
| Same shape, different coords, `+`/`-` | Positional match (silent bug) | `ValueError` |
| Different shape, `+`/`-` | `"outer"` or `"left"` (implicit) | `ValueError` |
| Mismatched coords, `*`/`/` | Crash or garbage | Intersection (or error if empty) |
| Constraint with mismatched RHS | `"override"` or `"left"` | `ValueError` |

To migrate: replace `x + y` with `x.add(y, join="outer")` (or whichever join matches your intent).